# Neural network model

Since we are working with text, we choose to train reccurent neural network, LSTM. 
Our architecture can be described as many-to-one - for many words on input we need to produce one label - 1 for positive and 0 for negative sentiment. 

Detailed architecture is explained further. 

In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras

In [2]:
import os

In [3]:
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout
from tensorflow.keras.regularizers import l1

In [5]:
%load_ext tensorboard
%tensorboard --logdir logs --bind_all

Reusing TensorBoard on port 6006 (pid 14156), started 1:16:28 ago. (Use '!kill 14156' to kill it.)

In [4]:
# load data
train = pd.read_pickle('../data/train/comments_embed.pkl')
test = pd.read_pickle('../data/test/comments_embed.pkl')

In [5]:
# prepare for training
train.head(n=5)

,comment,sentiment,comment_ids,comment_text,words_n,x
0,"[bromwel, high, cartoon, comedi, ran, the, sam...",1,"[336, 809, 171, 1915, 1, 148, 27, 26, 48, 1378...",bromwel high cartoon comedi ran the same time ...,98,"[336, 809, 171, 1915, 1, 148, 27, 26, 48, 1378..."
1,"[homeless, houseless, georg, carlin, state, be...",1,"[2736, 658, 12488, 591, 60, 835, 8, 96, 10, 97...",homeless houseless georg carlin state been iss...,310,"[2736, 658, 12488, 591, 60, 835, 8, 96, 10, 97..."
2,"[brilliant, overact, lesley, ann, warren, best...",1,"[523, 2081, 13669, 1105, 3199, 101, 780, 11953...",brilliant overact lesley ann warren best drama...,112,"[523, 2081, 13669, 1105, 3199, 101, 780, 11953..."
3,"[thi, easili, the, most, underr, film, inn, th...",1,"[3, 686, 1, 72, 1846, 7, 8550, 1, 1994, 3971, ...",thi easili the most underr film inn the brook ...,93,"[3, 686, 1, 72, 1846, 7, 8550, 1, 1994, 3971, ..."
4,"[thi, not, the, typic, mel, brook, film, much,...",1,"[3, 5, 1, 668, 3125, 1994, 7, 56, 341, 2303, 5...",thi not the typic mel brook film much less sla...,95,"[3, 5, 1, 668, 3125, 1994, 7, 56, 341, 2303, 5..."


In [6]:
train.x[0]

array([  336,   809,   171,  1915,     1,   148,    27,    26,    48,
        1378,    23,   391,   102,   119,  1359,    96,     1,  1555,
        3746,   238,   154,     4,   336,  1348,    56,  2128,   596,
          53,  1359,     1,  8985,   933,  2800,     1,  1533,   813,
          17,    29,    37,   176,   124,    47,  1108,  1359, 14390,
           1,  3939,     1,   208,   519,    15,   659,     1,   391,
         673,     2,    47,   813,    32,   199,     1,   243,    42,
         813,  3106,   100,  1015,   167,     1,   391,  1021,  1677,
         336,   299,   191,  2523,   105,  5439,    14,    82,  1359,
         813,  1782,   336,   212,     4,    90,   697,   419,    68,
           4,   336,   222,  4349,    25,  1574,     4,     5,     0,
           0])

In [7]:
test.head(n=5)

,comment,sentiment,comment_ids,x
0,"[went, and, saw, thi, movi, last, night, after...",1,"[440, 2, 199, 3, 6, 201, 284, 83, 10196, 147, ...","[440, 2, 199, 3, 6, 201, 284, 83, 10196, 147, ..."
1,"[actor, turn, director, bill, paxton, follow, ...",1,"[93, 157, 127, 769, 4320, 279, 949, 1660, 1, 8...","[93, 157, 127, 769, 4320, 279, 949, 1660, 1, 8..."
2,"[recreat, golfer, with, some, knowledg, the, s...",1,"[2592, 9, 26, 1620, 1, 1391, 478, 512, 9, 803,...","[2592, 9, 26, 1620, 1, 1391, 478, 512, 9, 803,..."
3,"[saw, thi, film, sneak, preview, and, delight,...",1,"[199, 3, 7, 2823, 2440, 2, 995, 1, 627, 1413, ...","[199, 3, 7, 2823, 2440, 2, 995, 1, 627, 1413, ..."
4,"[bill, paxton, taken, the, true, stori, the, g...",1,"[769, 4320, 623, 1, 273, 40, 1, 6595, 318, 2, ...","[769, 4320, 623, 1, 273, 40, 1, 6595, 318, 2, ..."


From the previous script, we know that our vocab contains 15000 words and max length of our comment is 100. 
We also choose our embedding size to be 100 for now - however, these are the hyper-parameters to played with later.

In [8]:
COMMENT_SIZE = 100
VOCAB_SIZE = 15000
EMBEDDING_SIZE = 100

Since we have pandas dataframe, structure of our data is np.array of np.arrays (not np.ndarray). 
This might cause problems when training - we need to explicitely convert it to 2d array - one way is using np.stack:

In [9]:
# no good, we need shappe (25000, 100)
train.x.values.shape

(25000,)

In [10]:
train_x = np.stack(train.x.values)

In [11]:
# ok
train_x.shape

(25000, 100)

In [12]:
test_x = np.stack(test.x.values)

In [13]:
# target (to make sure we have np arrays)
train_y = np.array(train.sentiment.values)
test_y = np.array(test.sentiment.values)

Our first neural network consists of layers:
- Embedding layer (to train basic word embedding fror NN to work with) (later we will compare with pretrained embeddings (or train our own embeddings))
- Bidirectional LSTM layer (we needed recurrent NN since we work with sequential data - text - so we chose LSTM. WE also went for Bidirectional since we read that it is capable of better understanding of context when making predictions - but there is also a potential to try and use other different architectures. )
Size of LSTM layer is also parametrizable - we can try different sizes and compare results - we will start with 64. 
- Since we need one number at the end - either 1 or 0 (positive or negative sentiment), we needed to add Dense layer to transform our result to such number. For activation function, we chose sigmoid (we were thinking about softmax, but since softmax is just generalized sigmoig (and usable for multiclass classification), we stayed with sigmoid in our problem)

Our first NN might be prone to overfitting. In future, we can add for example Dropout layer to try to prevent overfitting. 

In [17]:
# define NN architecture
class SentimentClassifier_v1(keras.Model):

    def __init__(self, vocab_size, embedding_size, comment_size, lstm_size):
        super(SentimentClassifier_v1, self).__init__()
        
        # train embedding 
        self.emb = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_size,
            input_length=comment_size,
            mask_zero=True, 
            trainable=True
        )
    
        self.lstm_layer = Bidirectional(LSTM(lstm_size))
        self.output_layer = Dense(1, activation="sigmoid")

    def call(self, x):
        x = self.emb(x)
        x = self.lstm_layer(x)
        x = self.output_layer(x)

        return x

In [18]:
# create NN object
nn_v1 = SentimentClassifier_v1(VOCAB_SIZE + 1, EMBEDDING_SIZE, COMMENT_SIZE, 64)

Before compiling our model, we need to choose optimizer. 

For the first try, we will go with Adam. Next we can try others like SGD.
Our loss function is now binary_crossentropy.

Our metrics is accuracy. We have balanced dataset (the same number of positive and negative classes) and in such case we think it is an ok metrics. 

In [19]:
# add callbacks - tensorboard and compile
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=os.path.join("logs", "sentiment_classifier_v1"),
        histogram_freq=1,
        profile_batch=0
    )
]

nn_v1.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

... aaand it is time for training!

In [20]:
nn_v1.fit(
    x=train_x,
    y=train_y,
    batch_size=32,
    epochs=10,
    validation_data=(test_x, test_y),
    callbacks=callbacks
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 102s 4ms/sample - loss: 0.3703 - accuracy: 0.8371 - val_loss: 0.3541 - val_accuracy: 0.8443
Epoch 2/10
25000/25000 [==============================] - 109s 4ms/sample - loss: 0.2169 - accuracy: 0.9174 - val_loss: 0.3734 - val_accuracy: 0.8332
Epoch 3/10
25000/25000 [==============================] - 102s 4ms/sample - loss: 0.1251 - accuracy: 0.9545 - val_loss: 0.4537 - val_accuracy: 0.8194
Epoch 4/10
25000/25000 [==============================] - 106s 4ms/sample - loss: 0.0588 - accuracy: 0.9799 - val_loss: 0.6286 - val_accuracy: 0.8176
Epoch 5/10
25000/25000 [==============================] - 103s 4ms/sample - loss: 0.0300 - accuracy: 0.9899 - val_loss: 0.7007 - val_accuracy: 0.8074
Epoch 6/10
25000/25000 [==============================] - 92s 4ms/sample - loss: 0.0208 - accuracy: 0.9934 - val_loss: 1.0269 - val_accuracy: 0.8202
Epoch 7/10
25000/25000 [===========================

Evaluation:
accuracy on valid: 0.80 afrer 9th epoch

Altough our first neural network seems to achieve quite fair results, based on the values of train and valid accuracy over the epochs we can tell that our NN is overfitting. 

We can try to add the Dropout layer to try to reduce overfitting. 
Other option, how to prevent overfitting, might be to use activity regularizer - l1/l2... we might also think about adding bias to our data - or to even decrease the complexity of network again. We also haven't preformed the hyperparameter tuning yet - which might help to achieve better results as well. 

And maybe try direct instead of bidirectional LSTM layer. 


In [21]:
# define NN architecture
class SentimentClassifier_v2(keras.Model):

    def __init__(self, vocab_size, embedding_size, comment_size, lstm_size):
        super(SentimentClassifier_v2, self).__init__()
        
        # train embedding 
        self.emb = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_size,
            input_length=comment_size,
            mask_zero=True, 
            trainable=True
        )
    
        self.lstm_layer = Bidirectional(LSTM(lstm_size, activity_regularizer=l1(0.001)))
        self.drouput_layer = Dropout(0.5)
        self.output_layer = Dense(1, activation="sigmoid")

    def call(self, x):
        x = self.emb(x)
        x = self.lstm_layer(x)
        x = self.output_layer(x)

        return x

In [22]:
# create NN object
nn_v2 = SentimentClassifier_v2(VOCAB_SIZE + 1, EMBEDDING_SIZE, COMMENT_SIZE, 64)

In [23]:
# compile and train
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=os.path.join("logs", "sentiment_classifier_v2"),
        histogram_freq=1,
        profile_batch=0
    )
]

nn_v2.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

nn_v2.fit(
    x=train_x,
    y=train_y,
    batch_size=32,
    epochs=15,
    validation_data=(test_x, test_y),
    callbacks=callbacks
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 100s 4ms/sample - loss: 0.3802 - accuracy: 0.8374 - val_loss: 0.3615 - val_accuracy: 0.8470
Epoch 2/15
25000/25000 [==============================] - 84s 3ms/sample - loss: 0.2177 - accuracy: 0.9214 - val_loss: 0.3904 - val_accuracy: 0.8326
Epoch 3/15
25000/25000 [==============================] - 84s 3ms/sample - loss: 0.1246 - accuracy: 0.9621 - val_loss: 0.4751 - val_accuracy: 0.8192
Epoch 4/15
25000/25000 [==============================] - 83s 3ms/sample - loss: 0.0678 - accuracy: 0.9828 - val_loss: 0.5505 - val_accuracy: 0.8059
Epoch 5/15
25000/25000 [==============================] - 84s 3ms/sample - loss: 0.0384 - accuracy: 0.9936 - val_loss: 0.7932 - val_accuracy: 0.8080
Epoch 6/15
25000/25000 [==============================] - 105s 4ms/sample - loss: 0.0268 - accuracy: 0.9973 - val_loss: 0.8514 - val_accuracy: 0.8059
Epoch 7/15
25000/25000 [==============================

Evaluation: we keep around 80% accuracy on valid data set. 

Again, we can see that our baseline model is not perfect and overfits. Over epochs, validation loss gets higher (whilst train loss decreases). This is something that we will try to make better during next project iterations. 

### Iteration 2.  - custom embedding

In [14]:
embedding = np.load('word_embeddings.npy')

In [16]:
COMMENT_SIZE = 100
VOCAB_SIZE = 15000
EMBEDDING_SIZE = 100

In [28]:
# define NN architecture
class SentimentClassifier_v3(keras.Model):

    def __init__(self, vocab_size, embedding_size, embedding_matrix, comment_size, lstm_size):
        super(SentimentClassifier_v3, self).__init__()
        
        # train embedding 
        self.emb = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_size,
            weights=[embedding_matrix],
            input_length=comment_size,
            mask_zero=True, 
            trainable=False
        )
    
        self.lstm_layer = Bidirectional(LSTM(lstm_size, activity_regularizer=l1(0.001)))
        self.drouput_layer = Dropout(0.5)
        self.output_layer = Dense(1, activation="sigmoid")

    def call(self, x):
        x = self.emb(x)
        x = self.lstm_layer(x)
        x = self.output_layer(x)

        return x

In [29]:
# create NN object
nn_v2 = SentimentClassifier_v3(VOCAB_SIZE + 1, EMBEDDING_SIZE, embedding, COMMENT_SIZE, 64)

In [30]:
# compile and train
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=os.path.join("logs", "sentiment_classifier_v3"),
        histogram_freq=1,
        profile_batch=0
    )
]

    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

nn_v2.fit(
    x=train_x,
    y=train_y,
    batch_size=32,
    epochs=15,
    validation_data=(test_x, test_y),
    callbacks=callbacks
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 96s 4ms/sample - loss: 0.6623 - accuracy: 0.6028 - val_loss: 0.6398 - val_accuracy: 0.6359
Epoch 2/15
25000/25000 [==============================] - 77s 3ms/sample - loss: 0.5965 - accuracy: 0.6794 - val_loss: 0.6070 - val_accuracy: 0.6602
Epoch 3/15
25000/25000 [==============================] - 77s 3ms/sample - loss: 0.5381 - accuracy: 0.7314 - val_loss: 0.5260 - val_accuracy: 0.7376
Epoch 4/15
25000/25000 [==============================] - 77s 3ms/sample - loss: 0.4860 - accuracy: 0.7677 - val_loss: 0.5746 - val_accuracy: 0.6995
Epoch 5/15
25000/25000 [==============================] - 80s 3ms/sample - loss: 0.4402 - accuracy: 0.7976 - val_loss: 0.4731 - val_accuracy: 0.7764
Epoch 6/15
25000/25000 [==============================] - 78s 3ms/sample - loss: 0.4069 - accuracy: 0.8176 - val_loss: 0.4816 - val_accuracy: 0.7641
Epoch 7/15
25000/25000 [==============================] 

### Trying LSTM layer instead of bidirectional

In [18]:
embedding = np.load('word_embeddings.npy')

In [19]:
COMMENT_SIZE = 100
VOCAB_SIZE = 15000
EMBEDDING_SIZE = 100

In [25]:
# define NN architecture
class SentimentClassifier_v4(keras.Model):

    def __init__(self, vocab_size, embedding_size, embedding_matrix, comment_size, lstm_size):
        super(SentimentClassifier_v4, self).__init__()
        
        # train embedding 
        self.emb = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_size,
            weights=[embedding_matrix],
            input_length=comment_size,
            mask_zero=True, 
            trainable=False
        )
    
        self.lstm_layer = LSTM(lstm_size, activity_regularizer=l1(0.001))
        self.drouput_layer = Dropout(0.5)
        self.output_layer = Dense(1, activation="sigmoid")

    def call(self, x):
        x = self.emb(x)
        x = self.lstm_layer(x)
        x = self.output_layer(x)

        return x

In [26]:
# create NN object
nn_v4 = SentimentClassifier_v4(VOCAB_SIZE + 1, EMBEDDING_SIZE, embedding, COMMENT_SIZE, 64)

In [33]:
# compile and train
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=os.path.join("logs", "sentiment_classifier_v4"),
        histogram_freq=1,
        profile_batch=0
    )
]

nn_v4.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

nn_v4.fit(
    x=train_x,
    y=train_y,
    batch_size=32,
    epochs=15,
    validation_data=(test_x, test_y),
    callbacks=callbacks
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 61s 2ms/sample - loss: 0.4596 - accuracy: 0.7932 - val_loss: 0.4719 - val_accuracy: 0.7965
Epoch 2/15
25000/25000 [==============================] - 51s 2ms/sample - loss: 0.4031 - accuracy: 0.8254 - val_loss: 0.4761 - val_accuracy: 0.7720
Epoch 3/15
25000/25000 [==============================] - 49s 2ms/sample - loss: 0.3911 - accuracy: 0.8326 - val_loss: 0.4242 - val_accuracy: 0.7987
Epoch 4/15
25000/25000 [==============================] - 51s 2ms/sample - loss: 0.3689 - accuracy: 0.8424 - val_loss: 0.4540 - val_accuracy: 0.8152
Epoch 5/15
25000/25000 [==============================] - 49s 2ms/sample - loss: 0.3556 - accuracy: 0.8493 - val_loss: 0.4592 - val_accuracy: 0.8085
Epoch 6/15
25000/25000 [==============================] - 48s 2ms/sample - loss: 0.3460 - accuracy: 0.8534 - val_loss: 0.3586 - val_accuracy: 0.8410
Epoch 7/15
25000/25000 [==============================] 

### Trying more epochs

In [15]:
embedding = np.load('word_embeddings.npy')

In [16]:
COMMENT_SIZE = 100
VOCAB_SIZE = 15000
EMBEDDING_SIZE = 100

In [17]:
# define NN architecture
class SentimentClassifier_v5(keras.Model):

    def __init__(self, vocab_size, embedding_size, embedding_matrix, comment_size, lstm_size):
        super(SentimentClassifier_v5, self).__init__()
        
        # train embedding 
        self.emb = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_size,
            weights=[embedding_matrix],
            input_length=comment_size,
            mask_zero=True, 
            trainable=False
        )
    
        self.lstm_layer = LSTM(lstm_size, activity_regularizer=l1(0.001))
        self.drouput_layer = Dropout(0.5)
        self.output_layer = Dense(1, activation="sigmoid")

    def call(self, x):
        x = self.emb(x)
        x = self.lstm_layer(x)
        x = self.output_layer(x)

        return x

In [18]:
# create NN object
nn_v5 = SentimentClassifier_v5(VOCAB_SIZE + 1, EMBEDDING_SIZE, embedding, COMMENT_SIZE, 64)

In [19]:
# compile and train
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=os.path.join("logs", "sentiment_classifier_v5"),
        histogram_freq=1,
        profile_batch=0
    )
]

nn_v5.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

nn_v5.fit(
    x=train_x,
    y=train_y,
    batch_size=32,
    epochs=30,
    validation_data=(test_x, test_y),
    callbacks=callbacks
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/30
25000/25000 [==============================] - 46s 2ms/sample - loss: 0.4662 - accuracy: 0.7872 - val_loss: 0.4287 - val_accuracy: 0.8131
Epoch 2/30
25000/25000 [==============================] - 40s 2ms/sample - loss: 0.4023 - accuracy: 0.8247 - val_loss: 0.4283 - val_accuracy: 0.8092
Epoch 3/30
25000/25000 [==============================] - 41s 2ms/sample - loss: 0.3823 - accuracy: 0.8356 - val_loss: 0.4186 - val_accuracy: 0.8072
Epoch 4/30
25000/25000 [==============================] - 41s 2ms/sample - loss: 0.3668 - accuracy: 0.8436 - val_loss: 0.3631 - val_accuracy: 0.8410
Epoch 5/30
25000/25000 [==============================] - 42s 2ms/sample - loss: 0.3559 - accuracy: 0.8472 - val_loss: 0.3741 - val_accuracy: 0.8400
Epoch 6/30
25000/25000 [==============================] - 41s 2ms/sample - loss: 0.3411 - accuracy: 0.8532 - val_loss: 0.3656 - val_accuracy: 0.8332
Epoch 7/30
25000/25000 [==============================] 

### v6 bidirectional layer with forward and backward layer

In [20]:
embedding = np.load('word_embeddings.npy')

In [21]:
COMMENT_SIZE = 100
VOCAB_SIZE = 15000
EMBEDDING_SIZE = 100

In [37]:
# define NN architecture
class SentimentClassifier_v6(keras.Model):

    def __init__(self, vocab_size, embedding_size, embedding_matrix, comment_size, lstm_size):
        super(SentimentClassifier_v6, self).__init__()
        
        # train embedding 
        self.emb = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_size,
            weights=[embedding_matrix],
            input_length=comment_size,
            mask_zero=True, 
            trainable=False
        )
    
        self.forward_layer = LSTM(lstm_size)
        self.backward_layer = LSTM(lstm_size, activation='relu', go_backwards=True)
        self.lstm_layer = Bidirectional(self.forward_layer, backward_layer=self.backward_layer)
        self.drouput_layer = Dropout(0.5)
        self.output_layer = Dense(1, activation="sigmoid")

    def call(self, x):
        x = self.emb(x)
        x = self.lstm_layer(x)
        x = self.output_layer(x)

        return x

In [38]:
# create NN object
nn_v6 = SentimentClassifier_v6(VOCAB_SIZE + 1, EMBEDDING_SIZE, embedding, COMMENT_SIZE, 64)

In [39]:
# compile and train
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=os.path.join("logs", "sentiment_classifier_v6"),
        histogram_freq=1,
        profile_batch=0
    )
]

nn_v6.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

nn_v6.fit(
    x=train_x,
    y=train_y,
    batch_size=32,
    epochs=30,
    validation_data=(test_x, test_y),
    callbacks=callbacks
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/30
25000/25000 [==============================] - 64s 3ms/sample - loss: 0.6362 - accuracy: 0.7416 - val_loss: 0.6379 - val_accuracy: 0.6871
Epoch 2/30
25000/25000 [==============================] - 58s 2ms/sample - loss: 13731.9157 - accuracy: 0.8107 - val_loss: 0.9882 - val_accuracy: 0.5123
Epoch 3/30
25000/25000 [==============================] - 57s 2ms/sample - loss: 0.4803 - accuracy: 0.7681 - val_loss: 0.4146 - val_accuracy: 0.8190
Epoch 4/30
25000/25000 [==============================] - 57s 2ms/sample - loss: 0.3771 - accuracy: 0.8330 - val_loss: 0.3824 - val_accuracy: 0.8286
Epoch 5/30
25000/25000 [==============================] - 58s 2ms/sample - loss: 0.3692 - accuracy: 0.8378 - val_loss: 0.3914 - val_accuracy: 0.8198
Epoch 6/30
25000/25000 [==============================] - 58s 2ms/sample - loss: 0.3597 - accuracy: 0.8395 - val_loss: 0.3610 - val_accuracy: 0.8408
Epoch 7/30
25000/25000 [============================

### v7 lstm_size  = 128

In [40]:
embedding = np.load('word_embeddings.npy')

In [41]:
COMMENT_SIZE = 100
VOCAB_SIZE = 15000
EMBEDDING_SIZE = 100

In [45]:
# define NN architecture
class SentimentClassifier_v7(keras.Model):

    def __init__(self, vocab_size, embedding_size, embedding_matrix, comment_size, lstm_size):
        super(SentimentClassifier_v7, self).__init__()
        
        # train embedding 
        self.emb = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_size,
            weights=[embedding_matrix],
            input_length=comment_size,
            mask_zero=True, 
            trainable=False
        )
    
        self.forward_layer = LSTM(lstm_size)
        self.backward_layer = LSTM(lstm_size, activation='relu', go_backwards=True)
        self.lstm_layer = Bidirectional(self.forward_layer, backward_layer=self.backward_layer)
        self.drouput_layer = Dropout(0.5)
        self.output_layer = Dense(1, activation="sigmoid")

    def call(self, x):
        x = self.emb(x)
        x = self.lstm_layer(x)
        x = self.output_layer(x)

        return x

In [46]:
# create NN object
nn_v7 = SentimentClassifier_v7(VOCAB_SIZE + 1, EMBEDDING_SIZE, embedding, COMMENT_SIZE, 128)

In [47]:
# compile and train
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=os.path.join("logs", "sentiment_classifier_v7"),
        histogram_freq=1,
        profile_batch=0
    )
]

nn_v7.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

nn_v7.fit(
    x=train_x,
    y=train_y,
    batch_size=32,
    epochs=30,
    validation_data=(test_x, test_y),
    callbacks=callbacks
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/30
25000/25000 [==============================] - 173s 7ms/sample - loss: 12130.2005 - accuracy: 0.6914 - val_loss: 0.4596 - val_accuracy: 0.7938
Epoch 2/30
25000/25000 [==============================] - 167s 7ms/sample - loss: 0.4326 - accuracy: 0.8099 - val_loss: 0.4037 - val_accuracy: 0.8226
Epoch 3/30
25000/25000 [==============================] - 227s 9ms/sample - loss: 0.4497 - accuracy: 0.8286 - val_loss: 0.3811 - val_accuracy: 0.8307
Epoch 4/30
25000/25000 [==============================] - 185s 7ms/sample - loss: 0.3667 - accuracy: 0.8389 - val_loss: 0.3813 - val_accuracy: 0.8384
Epoch 5/30
25000/25000 [==============================] - 187s 7ms/sample - loss: 0.3511 - accuracy: 0.8461 - val_loss: 0.3977 - val_accuracy: 0.8163
Epoch 6/30
25000/25000 [==============================] - 185s 7ms/sample - loss: 0.3360 - accuracy: 0.8548 - val_loss: 0.3608 - val_accuracy: 0.8411
Epoch 7/30
25000/25000 [======================

### v8 lstm_size = 32

In [48]:
embedding = np.load('word_embeddings.npy')

In [49]:
COMMENT_SIZE = 100
VOCAB_SIZE = 15000
EMBEDDING_SIZE = 100

In [50]:
# define NN architecture
class SentimentClassifier_v8(keras.Model):

    def __init__(self, vocab_size, embedding_size, embedding_matrix, comment_size, lstm_size):
        super(SentimentClassifier_v8, self).__init__()
        
        # train embedding 
        self.emb = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_size,
            weights=[embedding_matrix],
            input_length=comment_size,
            mask_zero=True, 
            trainable=False
        )
    
        self.forward_layer = LSTM(lstm_size)
        self.backward_layer = LSTM(lstm_size, activation='relu', go_backwards=True)
        self.lstm_layer = Bidirectional(self.forward_layer, backward_layer=self.backward_layer)
        self.drouput_layer = Dropout(0.5)
        self.output_layer = Dense(1, activation="sigmoid")

    def call(self, x):
        x = self.emb(x)
        x = self.lstm_layer(x)
        x = self.output_layer(x)

        return x

In [51]:
# create NN object
nn_v8 = SentimentClassifier_v8(VOCAB_SIZE + 1, EMBEDDING_SIZE, embedding, COMMENT_SIZE, 32)

In [52]:
# compile and train
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=os.path.join("logs", "sentiment_classifier_v8"),
        histogram_freq=1,
        profile_batch=0
    )
]

nn_v8.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

nn_v8.fit(
    x=train_x,
    y=train_y,
    batch_size=32,
    epochs=30,
    validation_data=(test_x, test_y),
    callbacks=callbacks
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/30
25000/25000 [==============================] - 107s 4ms/sample - loss: 0.4928 - accuracy: 0.7781 - val_loss: 0.4297 - val_accuracy: 0.8079
Epoch 2/30
25000/25000 [==============================] - 57s 2ms/sample - loss: 0.4044 - accuracy: 0.8200 - val_loss: 0.4044 - val_accuracy: 0.8104
Epoch 3/30
25000/25000 [==============================] - 61s 2ms/sample - loss: 0.3746 - accuracy: 0.8337 - val_loss: 0.3940 - val_accuracy: 0.8324
Epoch 4/30
25000/25000 [==============================] - 58s 2ms/sample - loss: 0.3590 - accuracy: 0.8439 - val_loss: 0.3686 - val_accuracy: 0.8358
Epoch 5/30
25000/25000 [==============================] - 60s 2ms/sample - loss: 0.3554 - accuracy: 0.8443 - val_loss: 0.4086 - val_accuracy: 0.8284
Epoch 6/30
25000/25000 [==============================] - 68s 3ms/sample - loss: 0.3428 - accuracy: 0.8516 - val_loss: 0.3551 - val_accuracy: 0.8421
Epoch 7/30
25000/25000 [==============================]

### v9 - long training, 100 epochs, learning_rate = 0.02, lstm_size = 16

In [53]:
embedding = np.load('word_embeddings.npy')

In [54]:
COMMENT_SIZE = 100
VOCAB_SIZE = 15000
EMBEDDING_SIZE = 100

In [57]:
# define NN architecture
class SentimentClassifier_v9(keras.Model):

    def __init__(self, vocab_size, embedding_size, embedding_matrix, comment_size, lstm_size):
        super(SentimentClassifier_v9, self).__init__()
        
        # train embedding 
        self.emb = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_size,
            weights=[embedding_matrix],
            input_length=comment_size,
            mask_zero=True, 
            trainable=False
        )
    
        self.forward_layer = LSTM(lstm_size)
        self.backward_layer = LSTM(lstm_size, activation='relu', go_backwards=True)
        self.lstm_layer = Bidirectional(self.forward_layer, backward_layer=self.backward_layer)
        self.drouput_layer = Dropout(0.5)
        self.output_layer = Dense(1, activation="sigmoid")

    def call(self, x):
        x = self.emb(x)
        x = self.lstm_layer(x)
        x = self.output_layer(x)

        return x

In [58]:
# create NN object
nn_v9 = SentimentClassifier_v9(VOCAB_SIZE + 1, EMBEDDING_SIZE, embedding, COMMENT_SIZE, 16)

In [60]:
# compile and train
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=os.path.join("logs", "sentiment_classifier_v9"),
        histogram_freq=1,
        profile_batch=0
    )
]

nn_v9.compile(
    learning_rate=0.02,
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

nn_v9.fit(
    x=train_x,
    y=train_y,
    batch_size=32,
    epochs=100,
    validation_data=(test_x, test_y),
    callbacks=callbacks
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/100
25000/25000 [==============================] - 145s 6ms/sample - loss: 16.1640 - accuracy: 0.7593 - val_loss: 0.4697 - val_accuracy: 0.7915
Epoch 2/100
25000/25000 [==============================] - 63s 3ms/sample - loss: 0.4000 - accuracy: 0.8236 - val_loss: 0.3981 - val_accuracy: 0.8198
Epoch 3/100
25000/25000 [==============================] - 63s 3ms/sample - loss: 0.3778 - accuracy: 0.8340 - val_loss: 0.4098 - val_accuracy: 0.8166
Epoch 4/100
25000/25000 [==============================] - 65s 3ms/sample - loss: 0.3668 - accuracy: 0.8371 - val_loss: 0.3680 - val_accuracy: 0.8379
Epoch 5/100
25000/25000 [==============================] - 62s 2ms/sample - loss: 0.3561 - accuracy: 0.8436 - val_loss: 0.3683 - val_accuracy: 0.8382
Epoch 6/100
25000/25000 [==============================] - 64s 3ms/sample - loss: 0.3468 - accuracy: 0.8507 - val_loss: 0.3590 - val_accuracy: 0.8415
Epoch 7/100
25000/25000 [=======================

Epoch 55/100
25000/25000 [==============================] - 51s 2ms/sample - loss: 0.1441 - accuracy: 0.9449 - val_loss: 0.5342 - val_accuracy: 0.8324
Epoch 56/100
25000/25000 [==============================] - 43s 2ms/sample - loss: 0.1431 - accuracy: 0.9459 - val_loss: 0.5361 - val_accuracy: 0.8311
Epoch 57/100
25000/25000 [==============================] - 45s 2ms/sample - loss: 0.1393 - accuracy: 0.9458 - val_loss: 0.5760 - val_accuracy: 0.8262
Epoch 58/100
25000/25000 [==============================] - 47s 2ms/sample - loss: 0.1424 - accuracy: 0.9454 - val_loss: 0.5970 - val_accuracy: 0.8225
Epoch 59/100
25000/25000 [==============================] - 61s 2ms/sample - loss: 0.1289 - accuracy: 0.9506 - val_loss: 0.5972 - val_accuracy: 0.8284
Epoch 60/100
25000/25000 [==============================] - 52s 2ms/sample - loss: 0.1260 - accuracy: 0.9527 - val_loss: 0.5851 - val_accuracy: 0.8302
Epoch 61/100
25000/25000 [==============================] - 51s 2ms/sample - loss: 0.1185 - ac

### v10 -  batch_size = 16 and learning_rate = 0.01

In [14]:
embedding = np.load('word_embeddings.npy')

In [15]:
COMMENT_SIZE = 100
VOCAB_SIZE = 15000
EMBEDDING_SIZE = 100

In [16]:
# define NN architecture
class SentimentClassifier_v10(keras.Model):

    def __init__(self, vocab_size, embedding_size, embedding_matrix, comment_size, lstm_size):
        super(SentimentClassifier_v10, self).__init__()
        
        # train embedding 
        self.emb = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_size,
            weights=[embedding_matrix],
            input_length=comment_size,
            mask_zero=True, 
            trainable=False
        )
    
        self.forward_layer = LSTM(lstm_size)
        self.backward_layer = LSTM(lstm_size, activation='relu', go_backwards=True)
        self.lstm_layer = Bidirectional(self.forward_layer, backward_layer=self.backward_layer)
        self.drouput_layer = Dropout(0.5)
        self.output_layer = Dense(1, activation="sigmoid")

    def call(self, x):
        x = self.emb(x)
        x = self.lstm_layer(x)
        x = self.output_layer(x)

        return x

In [17]:
# create NN object
nn_v10 = SentimentClassifier_v10(VOCAB_SIZE + 1, EMBEDDING_SIZE, embedding, COMMENT_SIZE, 16)

In [18]:
# compile and train
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=os.path.join("logs", "sentiment_classifier_v10"),
        histogram_freq=1,
        profile_batch=0
    )
]

nn_v10.compile(
    learning_rate=0.01,
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

nn_v10.fit(
    x=train_x,
    y=train_y,
    batch_size=16,
    epochs=30,
    validation_data=(test_x, test_y),
    callbacks=callbacks
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/30
25000/25000 [==============================] - 96s 4ms/sample - loss: 0.4668 - accuracy: 0.7894 - val_loss: 0.4344 - val_accuracy: 0.8003
Epoch 2/30
25000/25000 [==============================] - 81s 3ms/sample - loss: 0.4073 - accuracy: 0.8291 - val_loss: 0.4194 - val_accuracy: 0.8060
Epoch 3/30
25000/25000 [==============================] - 79s 3ms/sample - loss: 0.3675 - accuracy: 0.8398 - val_loss: 0.3714 - val_accuracy: 0.8324
Epoch 4/30
25000/25000 [==============================] - 85s 3ms/sample - loss: 0.3575 - accuracy: 0.8441 - val_loss: 0.3659 - val_accuracy: 0.8361
Epoch 5/30
25000/25000 [==============================] - 82s 3ms/sample - loss: 0.3474 - accuracy: 0.8466 - val_loss: 0.3913 - val_accuracy: 0.8295
Epoch 6/30
25000/25000 [==============================] - 86s 3ms/sample - loss: 0.3363 - accuracy: 0.8549 - val_loss: 0.3497 - val_accuracy: 0.8436
Epoch 7/30
25000/25000 [==============================] 

### v11 - added activity regulizer

In [19]:
embedding = np.load('word_embeddings.npy')

In [20]:
COMMENT_SIZE = 100
VOCAB_SIZE = 15000
EMBEDDING_SIZE = 100

In [25]:
# define NN architecture
class SentimentClassifier_v11(keras.Model):

    def __init__(self, vocab_size, embedding_size, embedding_matrix, comment_size, lstm_size):
        super(SentimentClassifier_v11, self).__init__()
        
        # train embedding 
        self.emb = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_size,
            weights=[embedding_matrix],
            input_length=comment_size,
            mask_zero=True, 
            trainable=False
        )
    
        self.forward_layer = LSTM(lstm_size, activity_regularizer=l1(0.001))
        self.backward_layer = LSTM(lstm_size, activation='relu', go_backwards=True, activity_regularizer=l1(0.001))
        self.lstm_layer = Bidirectional(self.forward_layer, backward_layer=self.backward_layer)
        self.drouput_layer = Dropout(0.5)
        self.output_layer = Dense(1, activation="sigmoid")

    def call(self, x):
        x = self.emb(x)
        x = self.lstm_layer(x)
        x = self.output_layer(x)

        return x

In [26]:
# create NN object
nn_v11 = SentimentClassifier_v11(VOCAB_SIZE + 1, EMBEDDING_SIZE, embedding, COMMENT_SIZE, 32)

In [27]:
# compile and train
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=os.path.join("logs", "sentiment_classifier_v11"),
        histogram_freq=1,
        profile_batch=0
    )
]

nn_v11.compile(
    learning_rate=0.03,
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

nn_v11.fit(
    x=train_x,
    y=train_y,
    batch_size=32,
    epochs=30,
    validation_data=(test_x, test_y),
    callbacks=callbacks
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/30
25000/25000 [==============================] - 76s 3ms/sample - loss: 0.6697 - accuracy: 0.7765 - val_loss: 0.4373 - val_accuracy: 0.8070
Epoch 2/30
25000/25000 [==============================] - 70s 3ms/sample - loss: 0.4062 - accuracy: 0.8234 - val_loss: 0.4198 - val_accuracy: 0.8122
Epoch 3/30
25000/25000 [==============================] - 70s 3ms/sample - loss: 0.3831 - accuracy: 0.8348 - val_loss: 0.4039 - val_accuracy: 0.8212
Epoch 4/30
25000/25000 [==============================] - 64s 3ms/sample - loss: 0.3707 - accuracy: 0.8410 - val_loss: 0.3865 - val_accuracy: 0.8334
Epoch 5/30
25000/25000 [==============================] - 68s 3ms/sample - loss: 0.3618 - accuracy: 0.8451 - val_loss: 0.3755 - val_accuracy: 0.8340
Epoch 6/30
25000/25000 [==============================] - 65s 3ms/sample - loss: 0.3543 - accuracy: 0.8489 - val_loss: 0.3632 - val_accuracy: 0.8412
Epoch 7/30
25000/25000 [==============================] 